<a href="https://colab.research.google.com/github/AmritSDutta/colab_ml/blob/main/train_own_voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install speechbrain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [25]:
import torch
from speechbrain.pretrained import EncoderClassifier
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
from transformers import SpeechT5HifiGan
from IPython.display import Audio
import numpy as np

# Load the x-vector model
# classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="tmpdir")
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="tmpdir")

# Load your audio
signal = classifier.load_audio('/content/sample_data/output.wav')

# Compute the embedding
embedding = classifier.encode_batch(signal)

print(embedding.shape)  # (batch, 512)

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Using symlink found at '/content/tmpdir/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Collecting files (or symlinks) for pretraining in tmpdir.
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Using symlink found at '/content/tmpdir/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /content/tmpdir/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Using symlink found at '/content/tmpdir/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /content/tmpdir/mean_var_norm_emb.ckpt
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Using symlink found at '/content/tmpdir/classifier.ckpt'
DEBUG:speechbrain.utils.paramete

torch.Size([1, 1, 512])


In [15]:
newText = '''
Meanwhile, in a parallel move to strengthen security measures,
Srinagar Police launched extensive searches across the city targeting
Over Ground Workers (OGWs) and terrorist associates linked to
banned outfits. Searches were conducted at the residences of 63 individuals
under proper legal protocols, with Executive Magistrates and independent
witnesses present. The operations, supervised by senior officers of the
Jammu and Kashmir Police, aimed to gather evidence, seize weapons
and digital devices, and dismantle networks conspiring against national security
'''

In [16]:
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

In [26]:
inputs = processor(text=newText, return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], embedding.squeeze(1), vocoder=vocoder)

In [27]:
Audio(speech, rate=16000)

In [28]:
import torchaudio

# Assume you already have speech tensor from model
if len(speech.shape) == 1:
    speech = speech.unsqueeze(0)

torchaudio.save("output_audio.wav", speech.cpu(), 16000)
print("Audio saved as output_audio.wav")

Audio saved as output_audio.wav


In [19]:
# Suppose 'embedding' is your (1, 512) torch.Tensor
# First, convert to numpy if needed
embedding_np = embedding.squeeze(1).cpu().numpy()

# Save it to disk
np.save("sample_speaker_embedding.npy", embedding_np)

print("Embedding saved!")

Embedding saved!
